In [ ]:
pip install tf-agents

     |████████████████████████████████| 1.2MB 5.3MB/s 


In [ ]:

from collections import deque
from collections import defaultdict
import pandas as pd
import numpy as np
import random
import keras
from tf_agents.networks import q_rnn_network
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
import tensorflow as tf
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
import os
import tempfile
from tf_agents.policies import policy_saver

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network
from tf_agents.utils import common
import sys
import matplotlib.pyplot as plt

In [ ]:
### reading data from csv and saving to dicts


days_prices = defaultdict(dict)
counter = 0
path = "/content/drive/MyDrive/Colab Notebooks/totrain"
for files in os.listdir(path):
    counter+=1
    np_dict = defaultdict(dict)
    prices = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/totrain/{files}", index_col = 0)
    prices = prices.drop("time", axis=1)
    l = len(prices.columns)
    for k in range(0,l):
        prices = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/totrain/{files}", index_col = 0)
        prices = prices.drop("time", axis=1)
        prices = prices.iloc[:, k:k+1]
        for i in prices.columns:
            prices[f"mov_5{i}"] = prices[f"{i}"].rolling(window=5).mean()
            prices[f"mov_20{i}"] = prices[f"{i}"].rolling(window=20).mean()
            prices[f"mov_80{i}"] = prices[f"{i}"].rolling(window=80).mean()
            prices = prices.dropna()
            prices["5-20"] = np.round((prices[f"mov_5{i}"] - prices[f"mov_20{i}"])/prices[f"{i}"], 4)
            prices["5-80"] = np.round((prices[f"mov_5{i}"] - prices[f"mov_80{i}"])/prices[f"{i}"], 4)
            prices["20-80"] = np.round((prices[f"mov_20{i}"] - prices[f"mov_80{i}"])/prices[f"{i}"], 4)
            np_dict[f"np_diff_{i}"] = np.array([prices["5-20"].values.reshape(-1,1), prices["5-80"].values.reshape(-1,1), prices["20-80"].values.reshape(-1,1), prices[f"{i}"].values.reshape(-1,1) ])
    days_prices[f"np_dict_{counter}"] = np_dict

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
### defining max length of training steps


def max_steps_lenght(days_prices):
    max_steps = 0
    for i in days_prices.keys():
        crypto_dict = days_prices[i].keys()
        number_of_crypto = len(crypto_dict)
        first_dict = list(crypto_dict)[0]
        lenght = len(days_prices[i]["np_diff_XLMUSDT"][0])-60
        lenght = lenght * (number_of_crypto) # timestamp
        max_steps += lenght
    return max_steps
max_steps_lenght(days_prices)

2279040

In [ ]:
### creating environment


class CustomEnv(py_environment.PyEnvironment):

    def __init__(self, days_price, initial_balance, lookback_window_size):

        self.lookback_window_size = lookback_window_size

        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')

        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(self.lookback_window_size * 3 + 1,), dtype=np.float32, name='observation')

        self.days_price = days_price

        self.days_count = 1 

        self.price_dict = days_price[f"np_dict_{self.days_count}"]

        self.dict_names = list(self.price_dict.keys())

        self.dict_count = 0

        self.initial_balance = initial_balance

        self.last_step = len(self.price_dict[f"{self.dict_names[self.dict_count]}"][0]) - self.lookback_window_size

        self.episode_ended = False

        self.active_position = 0

        self.action = 0

        self.fees = 0.001

        self.spread = 0.001

        self.money = initial_balance

        self.current_step = 0

        self.cum_reward = 0

        self.crypto_held = 0

        self.previous_money = initial_balance

        self._state = False
        
        self.price_bought = 0

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self.episode_ended = False
        self.current_step = 0
        self.price_bought = 0
        self.money = self.initial_balance
        self.active_position = 0
        self.action = 0
        self.crypto_held = 0
        self.dict_count += 1
        if self.dict_count == len(self.dict_names):
            self.days_count += 1
            self.dict_count = 0
            self.price_dict = self.days_price[f"np_dict_{self.days_count}"]
            self.dict_names = list(self.price_dict.keys())
            self.last_step = len(self.price_dict[f"{self.dict_names[self.dict_count]}"][0]) - self.lookback_window_size
        self.mov_price = self.price_dict[f"{self.dict_names[self.dict_count]}"]
        # try:
        #   self.mov_price = self.price_dict[f"{self.dict_names[self.dict_count]}"]
        # except IndexError:
        #     self.days_count += 1
        #     self.dict_count = 0
        #     self.price_dict = self.days_price[f"np_dict_{self.days_count}"]
        #     self.dict_names = list(self.price_dict.keys())
        #     self.last_step = len(self.price_dict[f"{self.dict_names[self.dict_count]}"][0]) - self.lookback_window_size
        self.previous_money = self.initial_balance
        self.cum_reward = 0  ###Only now
        self.pnl = 0

        self.last = self.current_step + self.lookback_window_size

        self._state = np.array(
            [self.mov_price[0, self.current_step:self.last], self.mov_price[1, self.current_step:self.last],
             self.mov_price[2, self.current_step:self.last]])

        self._state = np.append(self._state, self.active_position)

        return ts.restart(np.array(self._state, dtype=np.float32))

    def open_position(self, action, current_price):
        reward = 0
        if (self.active_position == 0):
            if action == 1:
                self.active_position = 1
                if self.money <= self.initial_balance * 0.8:
                    self.episode_ended = True
                    reward = -200
                else:
                    reward = 0
                    self.money = self.money - (self.money * self.spread) - (self.money * self.fees)
                    self.crypto_held = (self.money / current_price)
                    self.price_bought = current_price
                    
        elif (self.active_position == 1):
            if action == 0:
                self.active_position = 0
                self.money = self.crypto_held * current_price - (self.money * self.spread) - (self.money * self.fees)
                reward = (current_price - self.price_bought) * self.crypto_held * 2
                self.crypto_held = 0
                self.price_bought = 0
            else:
                self.money = self.crypto_held * current_price
                reward = 0

        return reward  # think about money balance

    def next_observation(self):

        self.last = self.current_step + self.lookback_window_size
        previous_price = self.mov_price[3, self.lookback_window_size + self.current_step - 1]
        current_price = self.mov_price[3, self.lookback_window_size + self.current_step]
        obs = np.array([self.mov_price[0, self.current_step:self.last], self.mov_price[1, self.current_step:self.last],
                        self.mov_price[2, self.current_step:self.last]])
        self.current_step += 1
        return obs, current_price, previous_price 

    def _step(self, action):
        reward = 0
        done = False
        if self.episode_ended == True:
            return self._reset()
        obs, current_price, previous_price = self.next_observation()
        self._state = obs
        if (action == 1):
            self.action = action
            reward = self.open_position(action, current_price)

        elif (action == 0):
            if self.active_position == 0:
                reward = -2
            else:
                self.action = action
                reward = self.open_position(action, current_price)

        if self.last == self.last_step: ##################################################################check should be if last = last.step -1
            self.episode_ended = True

        if self.episode_ended == True:
            done = True
            self.money = self.crypto_held * current_price - (self.money * self.spread) - (self.money * self.fees)
            self.active_position = 0
            reward = self.money - self.initial_balance

        pnl = (self.money - self.previous_money)
        self.previous_money = self.money
        self.pnl += pnl
        info = {}
        self._state = np.append(self._state, self.active_position)
        # return self._state, reward, done, info
        if self.episode_ended == True:
            return ts.termination(np.array(self._state, dtype=np.float32), float(reward))
        else:
            return ts.transition(
                np.array(self._state, dtype=np.float32), reward=float(reward), discount=0)

    def render(self):
        print(f'Step: {self.current_step}, Money: {self.money}, PnL{self.pnl}, Cum reward{self.cum_reward}')
    #####SKALIBROWAĆ REWARD




In [ ]:
#### creating model and agent


class Agent_net():

    def __init__(self, env):
        # defining hyperparameters
        self.env = env

        self.train_env = tf_py_environment.TFPyEnvironment(self.env)
        self.eval_env = tf_py_environment.TFPyEnvironment(self.env)
        # network configuration
        self.input_fc_layer_params = (181,)
        self.lstm_size = (60,)
        self.output_fc_layer_params = (60,)
        self.learning_rate = 0.005
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.td_errors_loss_fn = common.element_wise_squared_loss
        self.train_step_counter = tf.compat.v1.train.get_or_create_global_step() #tf.Variable(0)  
        self.replay_buffer_max_length = 1000000
        self.batch_size = 64
        self.epsilon = 0.2

    def qrnn(self):
        # create a q_RNNnet
        self.q_net = q_rnn_network.QRnnNetwork(
            self.train_env.observation_spec(),
            self.train_env.action_spec(),
            input_fc_layer_params=self.input_fc_layer_params,
            lstm_size=self.lstm_size,
            output_fc_layer_params=self.output_fc_layer_params)

    def agent_create(self):
        self.qrnn()
        self.target = self.qrnn()
        self.agent = dqn_agent.DqnAgent(
            self.train_env.time_step_spec(),
            self.train_env.action_spec(),
            q_network=self.q_net,
            #target_q_network = self.target,
            #target_update_tau = 0.1,
            #target_update_period = 100,
            epsilon_greedy = self.epsilon,
            optimizer=self.optimizer,
            td_errors_loss_fn=self.td_errors_loss_fn,
            train_step_counter=self.train_step_counter)

        self.agent.initialize() # just to start agent
        self.eval_policy = self.agent.policy # return the policy
        self.random_policy = random_tf_policy.RandomTFPolicy(self.train_env.time_step_spec(),
                                                             self.train_env.action_spec())

    def r_buffer(self):
        self.agent_create()
        self.replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
            data_spec=self.agent.collect_data_spec, # getting data scpecification 
            batch_size=self.train_env.batch_size, # how many experience there are in a batch
            max_length=self.replay_buffer_max_length) # maximum capacity of buffer is max_lenght * batch size (stored in a tuple)

    def iter_data(self):
        dataset = self.replay_buffer.as_dataset(
            num_parallel_calls=3, #how many process at time, if none sequential
            sample_batch_size=self.batch_size, #how many batches to get
            num_steps=10).prefetch(3) #how many steps in a batch

        iterator = iter(dataset)
        return (self.agent, self.train_env, self.replay_buffer, self.eval_env, iterator)

In [ ]:
### steps collection for training



def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step() # returns the current time_step and initializes the environment if needed
    action_step = policy.action(time_step, policy.get_initial_state(batch_size=env.batch_size)) #works with 1, Generates the initial state for stateful policies ?
    next_time_step = environment.step(action_step.action) # gets next time_step from taken action
    traj = trajectory.from_transition(time_step, action_step, next_time_step) # change it to trajectory, which represents a sequence of aligned time steps

    # Add trajectory to the replay buffer
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)


In [ ]:
### computing reward returned in order to work collect_step() must be first



def compute_avg_return(environment, policy, num_eval_episodes):

    total_return = 0.0
    for _ in range(num_eval_episodes): # number o evaluation episodes

        time_step = environment.reset() # getting first time_step
        episode_return = 0
        policy_state = policy.get_initial_state(batch_size=env.batch_size)
        avg_return = 0
        

        while not time_step.is_last():
            action_step = policy.action(time_step, policy_state) # action taken for given timestep
            time_step = environment.step(action_step.action) # assign next time_step 
            episode_return += time_step.reward # reward taken from next time_step ?
           
        total_return+= episode_return
        print(episode_return)
    avg_return = total_return  / num_eval_episodes
    return avg_return

In [ ]:
### training


def training(agent, train_env, replay_buffer, eval_env, iterator):
    num_iterations = 1000000  
    collect_steps_per_iteration = 10 
    log_interval = 5000
    eval_interval = 100000
    num_eval_episodes = 3
    index_out = False

    try:
        % % time # used for prolonged training
    except:
        pass

    # (Optional) Optimize by wrapping some of the code in a graph using TF function.
    agent.train = common.function(agent.train)

    # Evaluate the agent's policy once before training.
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes) #use created function
    returns = [avg_return]

    for _ in range(num_iterations):

        # Collect a few steps using collect_policy and save to the replay buffer.
        try:
          for _ in range(collect_steps_per_iteration):
              collect_step(train_env, agent.collect_policy, replay_buffer) # expanding buffer, collect_policy used to gather info target and local read more
        except IndexError:
          index_out = True   

        # Sample a batch of data from the buffer and update the agent's network.
        experience, unused_info = next(iterator) 
        train_loss = agent.train(experience).loss

        step = agent.train_step_counter.numpy()

        if step % log_interval == 0:
            print('step = {0}: loss = {1}'.format(step, train_loss))
            
        if step % eval_interval == 0:
            try:
                avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
                print('step = {0}: Average Return = {1}'.format(step, avg_return))
                returns.append(avg_return)
            except Exception:
                print("index error")
                index_out = True
        if index_out == True:
          break

    return agent.train_step_counter, agent, replay_buffer

In [ ]:
### calling env, agent creation, training etc

env = None
tf.compat.v1.reset_default_graph()
env = CustomEnv(days_prices, 1000, 60)
na = Agent_net(env)
na.r_buffer()
agent, train_env, replay_buffer, eval_env, iterator = na.iter_data()
save_train_step_counter, save_agent, save_replay_buffer = training(agent, train_env, replay_buffer, eval_env, iterator)

chkpdir = "/content/drive/MyDrive/Colab Notebooks/policies"
policy_dir = os.path.join(chkpdir, 'policy')
checkpoint_dir = os.path.join(chkpdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    model = na.q_net,
    agent=save_agent,
    policy=save_agent.policy,
    replay_buffer=save_replay_buffer,
    global_step=save_agent.train_step_counter
)

tf_policy_saver = policy_saver.PolicySaver(save_agent.policy)
checkpoint = train_checkpointer.save(save_agent.train_step_counter)
#train_checkpointer.initialize_or_restore()
#global_step = tf.compat.v1.train.get_global_step()
tf_policy_saver.save(policy_dir)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
tf.Tensor([68.26729], shape=(1,), dtype=float32)
tf.Tensor([3.737951], shape=(1,), dtype=float32)
tf.Tensor([27.077602], shape=(1,), dtype=float32)
step = 5000: loss = 26563.287109375
step = 10000: loss = 15929.7353515625
step = 15000: loss = 35056.953125
step = 20000: loss = 25634.2734375
step = 25000: loss = 520.248779296875
step = 30000: loss = 16435.771484375
step = 35000: loss = 19494.599609375
step = 40000: loss = 15192.013671875
step = 45000: loss = 15668.3125
step = 50000: loss = 171.50326538085938
step = 55000: loss = 143.80511474609375
step = 60000: loss = 5807.59619140625
step = 65000: loss = 15778.8369140625
step = 70000: loss = 15302.8271484375
step = 75000: loss = 268.3473815917969
step = 80000: loss = 15721.26171875
step = 85000: loss = 15862.2724609375
step = 90000: loss = 46505.32421875
step = 95000: loss = 259.28289794921875
step = 100000: loss = 15886.98046875
tf.Tensor([-2713.6

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/policies/policy/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/policies/policy/assets


In [ ]:
chkpdir = "/content/drive/MyDrive/Colab Notebooks/policies"
policy_dir = os.path.join(chkpdir, 'policy')
checkpoint_dir = os.path.join(chkpdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=agent.train_step_counter
)

tf_policy_saver = policy_saver.PolicySaver(agent.policy)
checkpoint = train_checkpointer.save(agent.train_step_counter)
#train_checkpointer.initialize_or_restore()
#global_step = tf.compat.v1.train.get_global_step()
tf_policy_saver.save(policy_dir)

In [ ]:
agent.train_step_counter

<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>